# Experiment 022: Solvent Similarity-Weighted Ensemble

**Based on evaluator feedback:**
- exp_021 (regularized) CV 0.0809 is 30% worse than best - DO NOT SUBMIT
- The CV-LB gap is NOT from overfitting - it's from test solvents being chemically different
- Need approaches that improve EXTRAPOLATION to unseen solvents

**Approach:**
1. Use exp_004 architecture (best CV 0.0623) as base
2. Add solvent similarity features using Spange descriptors
3. Ensemble multiple models with different random seeds for robustness

**TEMPLATE COMPLIANCE**: Last 3 cells are EXACTLY as template, NO cells after them.

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.metrics.pairwise import cosine_similarity
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = '/home/data'
torch.set_default_dtype(torch.double)
print("Setup complete")

Setup complete


In [2]:
# --- UTILITY FUNCTIONS ---
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load feature sets
SPANGE_DF = load_features('spange_descriptors')
ACS_PCA_DF = load_features('acs_pca_descriptors')
print(f"Spange: {SPANGE_DF.shape}, ACS_PCA: {ACS_PCA_DF.shape}")

Spange: (26, 13), ACS_PCA: (24, 5)


In [3]:
# --- BASE CLASSES ---
class SmilesFeaturizer(ABC):
    def __init__(self): raise NotImplementedError
    def featurize(self, X): raise NotImplementedError

class BaseModel(ABC):
    def __init__(self): pass
    def train_model(self, X_train, y_train): raise NotImplementedError
    def predict(self): raise NotImplementedError

In [4]:
# --- DUAL FEATURIZER (Spange + ACS_PCA) ---
class DualFeaturizer:
    """Featurizer that builds features from both Spange and ACS_PCA descriptors."""
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange = SPANGE_DF
        self.acs_pca = ACS_PCA_DF

    def _build_process_features(self, X):
        """Build process features with Arrhenius kinetics."""
        rt = X['Residence Time'].values.astype(np.float64).reshape(-1, 1)
        temp = X['Temperature'].values.astype(np.float64).reshape(-1, 1)
        
        # Arrhenius kinetic features
        temp_k = temp + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(rt + 1e-6)
        interaction = inv_temp * log_time
        
        return np.hstack([rt, temp, inv_temp, log_time, interaction])

    def _get_solvent_features(self, X, feature_df, flip=False):
        """Get solvent features from a feature dataframe."""
        if self.mixed:
            A = feature_df.loc[X["SOLVENT A NAME"]].values
            B = feature_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1) / 100.0
            if flip:
                return B * (1 - pct) + A * pct
            else:
                return A * (1 - pct) + B * pct
        else:
            return feature_df.loc[X["SOLVENT NAME"]].values

    def featurize_spange(self, X, flip=False):
        """Build features using Spange descriptors."""
        process = self._build_process_features(X)
        solvent = self._get_solvent_features(X, self.spange, flip)
        if self.mixed:
            pct = X["SolventB%"].values.reshape(-1, 1) / 100.0
            return np.hstack([process, pct, solvent])
        return np.hstack([process, solvent])

    def featurize_acs(self, X, flip=False):
        """Build features using ACS_PCA descriptors."""
        process = self._build_process_features(X)
        solvent = self._get_solvent_features(X, self.acs_pca, flip)
        if self.mixed:
            pct = X["SolventB%"].values.reshape(-1, 1) / 100.0
            return np.hstack([process, pct, solvent])
        return np.hstack([process, solvent])

In [5]:
# --- ENSEMBLE PER-TARGET MODEL (exp_004 architecture with multiple seeds) ---
class EnsemblePerTargetModel(BaseModel):
    """Per-target heterogeneous model with ensemble of multiple seeds.
    
    Uses exp_004 architecture (best CV 0.0623) with:
    - SM: HistGradientBoostingRegressor (max_depth=7, max_iter=700, lr=0.04)
    - Products: ExtraTreesRegressor (n_estimators=500, min_samples_leaf=2, max_depth=10)
    - Feature weighting: 0.8 acs_pca + 0.2 spange
    - Ensemble of 3 models with different random seeds
    """
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = DualFeaturizer(mixed=(data=='full'))
        self.targets = ['Product 2', 'Product 3', 'SM']
        
        # Scalers for each feature set
        self.scaler_spange = StandardScaler()
        self.scaler_acs = StandardScaler()
        
        # Models: {target: {feature_set: [models]}}
        self.models = {}
        
        # Feature weights: 0.8 acs_pca + 0.2 spange
        self.acs_weight = 0.8
        self.spange_weight = 0.2
        
        # Random seeds for ensemble
        self.seeds = [42, 123, 456]

    def train_model(self, X_train, y_train):
        # Build features
        X_spange = self.featurizer.featurize_spange(X_train)
        X_acs = self.featurizer.featurize_acs(X_train)
        y = y_train.values
        
        if self.data_type == 'full':
            # Data augmentation with flipped features
            X_spange_flip = self.featurizer.featurize_spange(X_train, flip=True)
            X_acs_flip = self.featurizer.featurize_acs(X_train, flip=True)
            X_spange = np.vstack([X_spange, X_spange_flip])
            X_acs = np.vstack([X_acs, X_acs_flip])
            y = np.vstack([y, y])
        
        # Scale features
        X_spange_scaled = self.scaler_spange.fit_transform(X_spange)
        X_acs_scaled = self.scaler_acs.fit_transform(X_acs)
        
        # Train per-target models with multiple seeds
        for i, target in enumerate(self.targets):
            self.models[target] = {'spange': [], 'acs': []}
            y_target = y[:, i]
            
            for seed in self.seeds:
                if target == 'SM':
                    model_spange = HistGradientBoostingRegressor(
                        max_depth=7, max_iter=700, learning_rate=0.04,
                        random_state=seed
                    )
                    model_acs = HistGradientBoostingRegressor(
                        max_depth=7, max_iter=700, learning_rate=0.04,
                        random_state=seed
                    )
                else:
                    model_spange = ExtraTreesRegressor(
                        n_estimators=500, min_samples_leaf=2, max_depth=10,
                        random_state=seed, n_jobs=-1
                    )
                    model_acs = ExtraTreesRegressor(
                        n_estimators=500, min_samples_leaf=2, max_depth=10,
                        random_state=seed, n_jobs=-1
                    )
                
                model_spange.fit(X_spange_scaled, y_target)
                model_acs.fit(X_acs_scaled, y_target)
                
                self.models[target]['spange'].append(model_spange)
                self.models[target]['acs'].append(model_acs)

    def predict(self, X):
        # Build features
        X_spange = self.featurizer.featurize_spange(X)
        X_acs = self.featurizer.featurize_acs(X)
        
        # Scale features
        X_spange_scaled = self.scaler_spange.transform(X_spange)
        X_acs_scaled = self.scaler_acs.transform(X_acs)
        
        # Predict per target with ensemble averaging
        predictions = []
        for target in self.targets:
            # Average predictions from all seeds
            pred_spange = np.mean([m.predict(X_spange_scaled) for m in self.models[target]['spange']], axis=0)
            pred_acs = np.mean([m.predict(X_acs_scaled) for m in self.models[target]['acs']], axis=0)
            
            # Weighted combination: 0.8 acs + 0.2 spange
            pred = self.acs_weight * pred_acs + self.spange_weight * pred_spange
            predictions.append(pred)
        
        # Stack and clip
        predictions = np.column_stack(predictions)
        predictions = np.clip(predictions, 0, 1)
        
        return torch.tensor(predictions)

print("EnsemblePerTargetModel defined")

EnsemblePerTargetModel defined


In [6]:
# Quick validation test
print("Testing EnsemblePerTargetModel...")
X_test, Y_test = load_data("single_solvent")

errors = []
for i, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_out_splits(X_test, Y_test)):
    if i >= 5: break
    model = EnsemblePerTargetModel(data='single')
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mae = np.mean(np.abs(preds - test_Y.values))
    errors.append(mae)
    solvent = test_X['SOLVENT NAME'].iloc[0]
    print(f"Single Fold {i} ({solvent}): MAE = {mae:.4f}")

print(f"\nEnsemble model quick test MAE: {np.mean(errors):.4f}")

Testing EnsemblePerTargetModel...


Single Fold 0 (1,1,1,3,3,3-Hexafluoropropan-2-ol): MAE = 0.1443


Single Fold 1 (2,2,2-Trifluoroethanol): MAE = 0.0962


Single Fold 2 (2-Methyltetrahydrofuran [2-MeTHF]): MAE = 0.0362


Single Fold 3 (Acetonitrile): MAE = 0.0821


Single Fold 4 (Acetonitrile.Acetic Acid): MAE = 0.1078

Ensemble model quick test MAE: 0.0933


In [7]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = EnsemblePerTargetModel(data='single') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:05,  5.67s/it]

2it [00:11,  5.85s/it]

3it [00:17,  5.82s/it]

4it [00:23,  5.84s/it]

...

21it [02:03,  5.83s/it]

22it [02:08,  5.83s/it]

23it [02:14,  5.81s/it]

24it [02:20,  5.81s/it]

24it [02:20,  5.86s/it]

In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = EnsemblePerTargetModel(data = 'full') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:07,  7.29s/it]

2it [00:14,  7.26s/it]

3it [00:22,  7.43s/it]

4it [00:29,  7.26s/it]

...

10it [01:11,  7.14s/it]

11it [01:19,  7.18s/it]

12it [01:26,  7.17s/it]

13it [01:33,  7.20s/it]

13it [01:33,  7.20s/it]

In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [10]:
# Calculate CV score - DO NOT include this cell in final submission
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Single solvent CV
single_preds = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
single_true = []
for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_out_splits(X_single, Y_single)):
    single_true.append(test_Y.values)
single_true = np.vstack(single_true)
single_mae = np.mean(np.abs(single_preds - single_true))
print(f"Single solvent CV MAE: {single_mae:.4f}")

# Full data CV
full_preds = submission_full_data[['target_1', 'target_2', 'target_3']].values
full_true = []
for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_ramp_out_splits(X_full, Y_full)):
    full_true.append(test_Y.values)
full_true = np.vstack(full_true)
full_mae = np.mean(np.abs(full_preds - full_true))
print(f"Full data CV MAE: {full_mae:.4f}")

# Combined
combined_mae = (single_mae + full_mae) / 2
print(f"\nCombined CV MAE: {combined_mae:.4f}")

Single solvent CV MAE: 0.0677
Full data CV MAE: 0.1126

Combined CV MAE: 0.0901
